#Unzip folder

In [ ]:
!unzip 'images.zip' -d images/

#Import Libraries

In [ ]:
import cv2 as cv
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


#Get Data

In [ ]:
# Crear las listas archivos de audio y su etiqueta correspondiente

from tensorflow.keras.utils import to_categorical
import glob
import os

def getExamples(datafolder):
  X_Image = []
  Y_Classification = []

  # Clasificaciones de clases
  image_classes = [os.path.basename(x) for x in glob.glob(datafolder + '*')]

  for i, image_class in enumerate(image_classes):
    for file in glob.glob(os.path.join(datafolder, image_class) + '/*.jpg'):
      X_Image.append(file)
      Y_Classification.append(np.array(to_categorical(i, num_classes=len(image_classes)),dtype=np.float32))
  return np.asarray(X_Image), np.asarray(Y_Classification)

In [ ]:
datafolder= '/content/images/'
X_Image, Y_Class = getExamples(datafolder)
print(len(X_Image), len(Y_Class))

In [ ]:
print(X_Image[147])
print(Y_Class[147])

In [ ]:
from sklearn.model_selection import train_test_split

X_Image, X_Image_test, Y_Class, Y_Class_test = train_test_split(X_Image, Y_Class, test_size=0.25)
print(len(X_Image))
print(len(Y_Class))

#Convert file paths to tensors

In [ ]:
def loadExample(example):
  # Cargar la imagen
  img = tf.io.read_file(example)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.rgb_to_grayscale(img);
  img = tf.image.convert_image_dtype(img, dtype=tf.float32)
  #img = img/255.0
  img = tf.reshape(img, [-1]);

  return img

In [ ]:
# Implementar el generador de Datos

class MySequence(tf.keras.utils.Sequence):

  def __init__(self, x_image, y_class, batch_size):
    self.x_image = x_image
    self.y_class = y_class
    self.batch_size = batch_size

  def __len__(self):
    return len(self.x_image)//self.batch_size

  def __getitem__(self, idx):

    batch_y = self.y_class[idx * self.batch_size : (idx+1)*self.batch_size]
    batch_x = np.zeros((self.batch_size, s.shape[0]), dtype=np.float32)
    for i in range(0, self.batch_size):
      batch_x[i] = loadExample(self.x_image[idx * self.batch_size + i])
    return batch_x, batch_y

In [ ]:
# Verificar la forma de los datos de entrada y la salida esperada

mS=MySequence(X_Image, Y_Class, 32)
my_data=iter(mS)
bx, by = next(my_data)
print(bx.shape, by.shape)

In [ ]:
bx.dtype

#Create a Dense Layer

In [ ]:
class Dense_Relu(tf.Module):
  #def __init__(self):
  def __init__(self, n_inputs, n_outputs, name=None):
    super().__init__(name=name)
    initial_w = tf.zeros(shape=[n_inputs, n_outputs], dtype=tf.float32)
    initial_b = tf.zeros(shape=[n_outputs], dtype=tf.float32)
    self.w = tf.Variable(initial_w)
    self.b = tf.Variable(initial_b)

  @tf.function
  def __call__(self, w):
    return tf.nn.relu(tf.matmul(w,self.w)  + self.b)

class Dense_SoftMax(tf.Module):
  #def __init__(self):
  def __init__(self, n_inputs, n_outputs, name=None):
    super().__init__(name=name)
    initial_w = tf.zeros(shape=[n_inputs, n_outputs], dtype=tf.float32)
    initial_b = tf.zeros(shape=[n_outputs], dtype=tf.float32)
    self.w = tf.Variable(initial_w)
    self.b = tf.Variable(initial_b)

  @tf.function
  def __call__(self, w):
    return tf.nn.softmax(tf.matmul(w,self.w) + self.b)

In [ ]:
class FC_Model(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.dense_1 = Dense_Relu(n_inputs=307200, n_outputs=10)
    self.dense_2 = Dense_SoftMax(n_inputs=10, n_outputs=2)
  @tf.function(input_signature=[tf.TensorSpec(shape=[None,307200], dtype=tf.float32)])
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)



#Watch Graph

In [ ]:
%load_ext tensorboard
from datetime import datetime
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/funcs/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = FC_Model()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model( tf.random.uniform(shape=[1,307200], minval=-1.0, maxval=1.0)))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)
%tensorboard --logdir logs/funcs

In [ ]:
new_model = FC_Model()

#Use Predefied Loss Function Categorical Cross Entropy

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy();

#Training

In [ ]:
epochs = 10

for epoch in range(epochs):
  my_data=iter(mS)
  for x_batch, y_batch in my_data:
    with tf.GradientTape() as tape:
      batch_loss = loss(new_model(x_batch), y_batch)

    # Update parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, new_model.variables)
    for g,v in zip(grads, new_model.variables):
        v.assign_sub(0.001*g)
  # Keep track of model loss per epoch
  loss_val = loss(new_model(x_batch), y_batch)
  print(f"Epoch {epoch} Loss  is {loss_val}",)

In [ ]:
def plotExample(example):
  # Cargar la imagen
  img = tf.io.read_file(example)
  img = tf.image.decode_jpeg(img, channels=3)
  plt.imshow(img)

In [ ]:
i=10
print(Y_Class_test[i])
print(X_Image_test[i])
pred = new_model([loadExample(X_Image_test[i])])
print(pred)
plotExample(X_Image_test[i])

In [ ]:
tf.saved_model.save(new_model, "/tmp/dense/")
converter = tf.lite.TFLiteConverter.from_saved_model("/tmp/dense/")

tflite_model = converter.convert()

with open('image_dense.tflite', 'wb') as f:
  f.write(tflite_model)